# Data Preparation

In [1]:
import sys
from copy import deepcopy
import warnings
import numpy as np
from numpy import inf, nan
import pandas as pd
import joblib
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
import featuretools as ft
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
current_directory = os.getcwd()
repertory_path = os.path.abspath(os.path.join(current_directory, "..", ".."))
sys.path.append("../../")

from insurance_charges_model.prediction.transformers import DFSTransformer
from insurance_charges_model.prediction.transformers import InfinityToNaNTransformer
from insurance_charges_model.prediction.transformers import IntToFloatTransformer
from insurance_charges_model.prediction.transformers import BooleanTransformer

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

/home/nicolas/.local/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


## Loading the Data

In [2]:
df = pd.read_csv(repertory_path + "/data/insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Deep Feature Synthesis
We'll be using the featuretools package to do feature engineering.

An EntitySet is an object that we will give to the featuretools package in order to do feature engineering. An entitySet denotes the features of specific "entity" in the real world. In this case, we will work with only one type of entity: "transactions".

In [3]:
entityset = ft.EntitySet(id="Transactions")
entityset = entityset.add_dataframe(dataframe_name="Transactions",
                                    dataframe=df,
                                    make_index=True,
                                    index="index")
entityset

Entityset: Transactions
  DataFrames:
    Transactions [Rows: 1338, Columns: 8]
  Relationships:
    No relationships

In [4]:
# Getting a list of variables associated with the EntitySet we just created
entityset["Transactions"].columns


Index(['index', 'age', 'sex', 'bmi', 'children', 'smoker', 'region',
       'charges'],
      dtype='object')

Now that we have defined an EntitySet for our data, we'll ask the featuretools package to create some features for us. The package defines a set of "primitives" that are able to create new features by processing the features that already exist in the EntitySet.

We are also going to ignore the categorical and boolean features in the dataset because they don't play well with the numerical features.

The feature_dataframe variable now contains the new features:

## Encode Features using Deep Feature Synthesis

Now we can create a Transformer that we can use later to create the features, given samples of the dataset.

In [5]:
from insurance_charges_model.prediction.transformers import DFSTransformer


In [6]:
target_entity = "Transactions"
trans_primitives = ["add_numeric", "subtract_numeric", "multiply_numeric", "divide_numeric", "greater_than", "less_than"]
ignore_variables = {"Transactions": ["sex", "smoker", "region"]}

dfs_transformer = DFSTransformer(target_entity=target_entity, trans_primitives=trans_primitives, ignore_variables=ignore_variables)


## Create Transformer for inf Values

Some of the features created by the featuretools package have a value of 'inf'. We'll create a transformer that maps these values to 0.0 to allow the models to be trained.

In [7]:
infinity_transformer = InfinityToNaNTransformer()

In [8]:
# testing the transformer
inpt = [[1.0], [inf], [1.0]]

# copying the transformer object in order to fit and test it
infinity_transformer_copy = deepcopy(infinity_transformer)

infinity_transformer_copy.fit(inpt)
result = infinity_transformer_copy.transform(inpt)

print(result)

[[ 1.]
 [nan]
 [ 1.]]


In order to handle the NaN values, we'll use a SimpleImputer that will fill in the missing value:

In [9]:
simple_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [10]:
# testing the transformer

# copying the transformer object in order to fit and test it
simple_imputer_copy = deepcopy(simple_imputer)

simple_imputer_copy.fit(result)

test_df = [[np.nan, 2, 3], [4, np.nan, 6], [10, np.nan, 9]]
print(simple_imputer_copy.transform(result))

[[1.]
 [1.]
 [1.]]


The SimpleImputer transformer has problems with imputing values that are not floats when using the 'mean' strategy. To fix this, we'll create a transformer that will convert all integer columns into floating point columns:

In [11]:
int_to_float_transformer = IntToFloatTransformer()

In [12]:
# test the transformer

# copying the transformer object in order to fit and test it
int_to_float_transformer_copy = deepcopy(int_to_float_transformer)

int_to_float_transformer_copy.fit(result)

test_df = [[2, 3.0], [4, 6.0], [10, 9.0]]
print(int_to_float_transformer_copy.transform(test_df))


[[ 2.  3.]
 [ 4.  6.]
 [10.  9.]]


Lastly, we'll put the IntToFloatTransformer, DFSTransformer, InfinityToNaNTransformer, and SimpleImputer transformers into a Pipeline so they'll all work together as a unit:

In [13]:
dfs_pipeline = Pipeline([
    ("dfs_transformer", dfs_transformer),
    ("int_to_float_transformer", int_to_float_transformer),
    ("infinity_transformer", infinity_transformer),
    ("simple_imputer", simple_imputer),
])


In [14]:
entityset = ft.EntitySet(id="Transactions")
entityset = entityset.add_dataframe(dataframe_name="Transactions",
                                    dataframe=df,
                                    make_index=True,
                                    index="index")


## Encode Boolean Features

We'll create a transformer that is able to convert the string in the 'smoker' feature to a boolean value.

In [15]:
boolean_transformer = BooleanTransformer(true_value="yes", false_value="no")

In [16]:
# testing the transformer
test_df = pd.DataFrame([["yes"], ["no"], ["yes"]],
                       columns=["smoker"])

# copying the transformer object in order to fit and test it
boolean_transformer_copy = deepcopy(boolean_transformer)

boolean_transformer_copy.fit(test_df)
result = boolean_transformer_copy.transform(test_df)

if (result != np.array([[True], [False], [True]])).all():
    raise ValueError("Unexpected values found in array.")

## Encode Categorical Features

Next, we'll create an encoder that will encode the categorical features. The categorical features that we will encode will be 'sex' and 'region'.

In [17]:
ordinal_encoder = OrdinalEncoder()

In [18]:
# testing the transformer
test_df = pd.DataFrame([["southwest"], ["northeast"], ["southwest"]],
                       columns=["region"])

# copying the transformer object in order to fit and test it
ordinal_encoder_copy = deepcopy(ordinal_encoder)

ordinal_encoder_copy.fit(test_df)
result = ordinal_encoder_copy.transform(test_df)

if (result != np.array([[1.0], [0.0], [1.0]])).all():
    raise ValueError("Unexpected values found in array.")

## Create ColumnTransformer

Combining all of the preprocessors into one ColumnTransformer that can be used to preprocess the data.

In [19]:
column_transformer = ColumnTransformer(
    remainder="passthrough",
    transformers=[
        ("dfs_pipeline", dfs_pipeline, ["age", "sex", "bmi", "children", "smoker", "region"]),
        ("boolean_transformer", boolean_transformer, ["smoker"]),
        ("ordinal_encoder", ordinal_encoder, ["sex", "region"])
    ]
)

In [20]:
from sklearn.base import BaseEstimator, TransformerMixin

class DFSTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, target_entity=None):
        self.target_entity = target_entity

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # your transformation code here
        pass


## Saving ColumnTransformer

NOTE: the ColumnTransformer object is saved in an UNFITTED state, it will be fitted to the data set later

In [21]:
joblib.dump(column_transformer, "transformer.joblib")

['transformer.joblib']